In [1]:
# Import libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dt_ids7_export_utils as bh_utils

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Remove unnessecary columns:
df_ids7 = bh_utils.remove_unnecessary_columns(df_ids7, True)
# Check for missing time data:
df_ids7 = bh_utils.filter_NaT(df_ids7, True)
# Check for cancelled procedures:
df_ids7 = bh_utils.filter_cancelled(df_ids7, True)
# Check for non-human subjects (e.g. phantoms, animals, etc.):
df_ids7 = bh_utils.filter_phantom_etc(df_ids7, True)
# Check for invalid accession numbers:
df_ids7 = bh_utils.check_accession_format(df_ids7, True)
# Check whether an accession number is in the DoseTrack data:
df_ids7 = bh_utils.check_accession_ids7_vs_dt(df_ids7, df_dt, True)
# Merge accession numbers performed in the same booking if one of the procedures is in dosetrack:
df_ids7 = bh_utils.merge_accession_no_same_procedure(df_ids7, df_dt, True)


Dropping unnecessary column: Prioritet- og lesemerkeikon
Dropping unnecessary column: Lagt til i demonstrasjon-ikon
Dropping unnecessary column: Status
Number of rows with NaT in the column "Bestilt dato og tidspunkt": 1
Number of cancelled procedures: 393
Number of non-human subjects: 7
Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2218
Number of accession numbers in IDS7 not in DoseTrack: 136
Patient: PAS0607, time: 2023-05-18 13:00:00, accession numbers: ['NRRH000008244733' 'NRRH000008244732']
Inserted accession number: NRRH000008244732 into patient: PAS0607, time: 2023-05-18 13:00:00, accession numbers: ['NRRH000008244733' 'NRRH000008244732']
Patient: PAS0835, time: 2023-05-04 15:00:00, accession numbers: ['NRRH000008224660' 'NRRH000008224398']
Patient: PAS1444, time: 2023-02-14 15:00:00, accession numbers: ['NRRH000008006267' 'NORRH00004327418']
Patient: PAS1517, time: 2023-02-06 15:00:00, accession numbers: ['NRRH000008000661' 'NORRH00004143

In [ ]:


# Go through all the patients in the IDS7 data:
patient_list = df_ids7['Pasient'].unique()
for patient in patient_list:
    # Go through all the individual booking times for this patient:
    booking_times = sorted(df_ids7[df_ids7['Pasient'] == patient]['Bestilt dato og tidspunkt'].unique())
    for time in booking_times:
        # Get the accession number for this patient at this booking time:
        acc_nr = df_ids7[(df_ids7['Pasient'] == patient) & (df_ids7['Bestilt dato og tidspunkt'] == time)]['Henvisnings-ID'].unique()
        # Check if there is more than one accession number for this patient at this time:
        if len(acc_nr) > 1:
            # Make a pandas series with accession numbers as index and true/false as values:
            print('Patient: ' + str(patient) + ', time: ' + str(time) + ', accession numbers: ' + str(acc_nr))
            acc_nr_in_dt = pd.Series(index=acc_nr, data = np.nan)
            # Go through all the accession numbers for this patient at this time:
            for acc in acc_nr:
                # Check if the accession number is in the DoseTrack data:
                acc_nr_in_dt[acc] = df_ids7[(df_ids7['Pasient'] == patient) & (df_ids7['Bestilt dato og tidspunkt'] == time) & (df_ids7['Henvisnings-ID'] == acc)]['Henvisning_i_dt'].unique()
            
            # Check if there are both true and false values:
            if acc_nr_in_dt.nunique() > 1:
                # Warn the user that there might be ambigous data regarding this procedure if there are several true and at least one false:
                if len(acc_nr_in_dt[acc_nr_in_dt == True]) > 1 and len(acc_nr_in_dt[acc_nr_in_dt == False]) > 0:
                    print('WARNING: there are two or more accessions with data in dosetrack and at least one without.')
                    print('Please investigate patient: ' + str(patient) + ', time: ' + str(time) + ', accession numbers: ' + str(acc_nr))
                else:
                    # Insert the accession number which is included in the DoseTrack data into all the rows for the same patient and booking with no dosetrack data:
                    df_ids7.loc[(df_ids7['Pasient'] == patient) & (df_ids7['Bestilt dato og tidspunkt'] == time) & (df_ids7['Henvisnings-ID'] == False) , 'Henvisning_i_dt'] = acc_nr_in_dt[acc_nr_in_dt == True].index[0]              
                    
                    



                
            

            


In [12]:
# Check if there are any rows with false values in the 'Henvisning_i_dt' column:
if df_ids7['Henvisning_i_dt'].any() == False:
    print('All accession numbers are in DoseTrack.')



We need to create logic for merging lines in the ids7 data:

During a booking the patient gets an accession number:
As the procedure moves forward more codes can be added to the same accession number. These codes should be merged into one line.

Check if the accession number exists in Dosetrack. If it does, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does not have a corresponding entry in dosetrack. If so they might be in the same procedure.

Alternatively, if there are an accession number that is not included in doseTrack, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does have a corresponding entry in dosetrack. If so they might be in the same procedure.

If neither of the procedures exists in dosetrack they might or might not be the same procedure.

Print a list of all the changes performed on the data.

In [ ]:
# Check for patients with multiple bookings on the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_time_with_different_accession(df_ids7)

In [6]:
# Check for patients with multiple bookings on the same day but not the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_day_with_different_accession(df_ids7)